# Lesson: Mounting an MCP Server in a FastAPI ASGI Application
Introduction & Overview

Welcome back! In the last two lessons, you learned how to connect your agent to one or more MCP servers, giving it access to a wide range of tools and services. You saw how tool caching can make your agent more efficient and how connecting to multiple MCP servers allows your agent to coordinate complex tasks — like managing a shopping list and placing orders — across different services.

In this lesson, we will take your skills a step further by showing you how to combine your MCP server with a modern web framework using ASGI. Specifically, you will learn how to mount your MCP server inside a FastAPI application. This approach lets you serve your MCP tools alongside other web endpoints, making your application more flexible and scalable. By the end of this lesson, you will be able to run your MCP server as part of a FastAPI app, ready to handle both standard API requests and agent tool calls — all in one place.
Overview of ASGI and FastAPI Essentials

Before we dive into the code, let’s quickly review what ASGI and FastAPI are and why they matter for your projects.

ASGI stands for Asynchronous Server Gateway Interface. It is a modern standard for Python web servers and applications, designed to support asynchronous programming. This means your application can handle many requests at the same time without waiting for each one to finish before starting the next. This is especially useful for real-time features, streaming, and high-concurrency workloads.

FastAPI is a popular ASGI framework that makes it easy to build fast, modern web APIs in Python. It is built on top of Starlette and uses Python’s async features to deliver high performance. FastAPI is known for its simple syntax, automatic documentation, and support for both synchronous and asynchronous code.

By combining your MCP server with FastAPI, you get the best of both worlds: the ability to serve your agent tools and regular web endpoints together, and the performance benefits of asynchronous programming.
FastAPI and Uvicorn Setup

If you are working on your own machine, you would typically install Uvicorn and FastAPI using pip:

Bash

pip install fastapi uvicorn

However, on CodeSignal, these libraries are already installed for you, so you can focus on writing and running your code without worrying about setup.
Mounting an MCP Server Within FastAPI

Let’s look at how you can mount your MCP server inside a FastAPI application. Below is a simplified version of the code you will work with:

Python

import uvicorn

from fastapi import FastAPI

from shopping_list import ShoppingListService

from mcp_server import mcp  # Import the MCP server with the shopping list tools


# Create the FastAPI app

app = FastAPI()


# Create a shopping list service instance

shopping_list = ShoppingListService()


# Regular FastAPI route for getting all shopping list items

@app.get("/items/")

async def get_all_items():

    """Get all shopping list items."""

    return shopping_list.get_items()


# Mount the MCP SSE app at '/'

app.mount('/', mcp.sse_app())

Here’s what’s happening in this code:

    First, we import the necessary modules, including FastAPI and our MCP server instance.

    Next, we create a FastAPI app.

    Then, we add a regular HTTP endpoint at /items/ that returns the current shopping list. This is just a typical web route in FastAPI — when someone visits /items/ in their browser or makes a request to that URL, they’ll get back the list of shopping items.

    The key part is the line app.mount('/', mcp.sse_app()). Here, we mount our MCP server’s SSE (Server-Sent Events) app at the root path of the FastAPI application. This means any requests to the root (such as /sse) will be handled by our MCP server, while other routes (like /items/) are handled by FastAPI.

The mcp.sse_app() method returns an ASGI-compatible application specifically designed to handle Server-Sent Events (SSE) for the MCP server. This application is not a FastAPI app but a standalone ASGI app built for streaming communication. By mounting it within the FastAPI application using app.mount('/', mcp.sse_app()), FastAPI transparently forwards requests to the MCP server’s SSE app, leveraging FastAPI’s ASGI-compliant structure. This integration allows you to combine the strengths of both frameworks: you can serve your MCP tools for real-time agent communication and also provide regular API endpoints, all within a single, unified application.
Running the Combined Application with Uvicorn

To run your combined FastAPI and MCP server application, you use Uvicorn, which is an ASGI server designed for high performance. In your code, you see the following block:

Python

if __name__ == "__main__":

    uvicorn.run(

        "main:app",

        host="0.0.0.0",

        port=3000,

        reload=True

    )

This tells Python to start the Uvicorn server, serving your FastAPI app on port 3000. The reload=True option is helpful during development, as it automatically restarts the server when you make changes to your code.

To start the server, you can simply run the following command from the directory containing your script:

Bash

python your_script.py

Once the server is running, you can access your regular API endpoints and your MCP server tools from the same application.
Agent Integration Example

Now, let’s see how an agent can connect to your MCP server running inside the FastAPI app. Here is an example agent script:

Python

import asyncio

from agents import Agent, Runner

from agents.mcp import MCPServerSse


async def main():

    # Setup the SSE server parameters

    server_params = {"url": "http://localhost:3000/sse"}


    # Connect to the MCP server over SSE

    async with MCPServerSse(params=server_params) as mcp_server:

        # Create an agent

        agent = Agent(

            name="OpenAI Shopping Agent",

            instructions="You are an assistant that uses shopping list tools to assist with a shopping list",

            mcp_servers=[mcp_server],

            model="gpt-4.1"

        )

        

        # Run the agent

        result = await Runner.run(

            starting_agent=agent,

            input="Give me my shopping list"

        )


        # Print the final output

        print(result.final_output)


if __name__ == "__main__":

    asyncio.run(main())

In this script, the client connects to your MCP server by establishing a real-time SSE (Server-Sent Events) connection to the endpoint you set up in your FastAPI app (for example, http://localhost:3000/sse). The MCPServerSse class manages this connection, allowing the agent and server to exchange messages directly.

Once the connection is established, the agent is given instructions to use the shopping list tools and processes the sample input. When the agent needs to use a tool (like getting the shopping list), it sends a request to the /messages endpoint on your server. FastAPI routes this request to the MCP server, which processes it and sends the response back to the agent over the SSE connection. This setup allows the agent to interact with your custom tools in real time and return the final result.
Mounting the MCP Server at a Custom Route

You have the flexibility to mount your MCP server at any route within your FastAPI application—not just at the root ('/'). This is an important architectural decision that can help you keep your application organized, avoid route conflicts, and apply different access controls as your project grows.

For example, mounting your MCP server at /mcp is a common choice to clearly separate MCP-related endpoints from your main API:

Python

# Mount the MCP SSE app at '/mcp'

app.mount('/mcp', mcp.sse_app())

With this configuration, all requests to /mcp and its subpaths (like /mcp/sse) are handled by your MCP server, while your other FastAPI endpoints (such as /items/) remain unaffected and accessible as usual.

Why is choosing a custom route important?

    Clear separation of concerns: By mounting at a dedicated path like /mcp, you keep your MCP endpoints distinct from your main API, making your application easier to maintain and reason about.
    Scalability and flexibility: As your application grows, this separation helps prevent route conflicts and allows you to expand your API surface without worrying about overlapping paths.
    Security and access control: You can apply specific authentication or authorization rules to the /mcp path, giving you fine-grained control over who can access your agent tools.

If you decide to mount your MCP server at a custom route, remember to update your client's connection URL accordingly:

Python

server_params = {"url": "http://localhost:3000/mcp/sse"}

This ensures the client connects to the correct endpoint for the MCP server. The handshake at /mcp/sse establishes a real-time connection, allowing the server to send events and instructions back to the agent. After the handshake, when the agent needs to use a tool, the client sends requests to the /mcp/messages endpoint as directed by the server. The MCP server processes these requests—such as adding an item to a shopping list or retrieving data—and sends the results back through the established SSE connection. This separation of endpoints keeps communication organized and efficient, and by mounting your MCP server at /mcp, your application structure remains clean and scalable for more complex scenarios.
Summary & Preparing for Practice

In this lesson, you learned how to mount your MCP server inside a FastAPI application, allowing you to serve both MCP tools and regular API endpoints from a single, scalable ASGI app. You saw how to use FastAPI’s app.mount() method to integrate your MCP server and how to run the combined application using Uvicorn. You also reviewed how an agent can connect to your MCP server through the mounted endpoint and complete real-world tasks.

This approach makes your applications more flexible and production-ready, as you can now combine agent tools and standard web APIs in one place. You are now ready to practice these skills in hands-on exercises. Great work progressing through this advanced integration technique!

In [ ]:
pip install fastapi uvicorn

In [ ]:
import uvicorn
from fastapi import FastAPI
from shopping_list import ShoppingListService
from mcp_server import mcp  # Import the MCP server with the shopping list tools

# Create the FastAPI app
app = FastAPI()

# Create a shopping list service instance
shopping_list = ShoppingListService()

# Regular FastAPI route for getting all shopping list items
@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

# Mount the MCP SSE app at '/'
app.mount('/', mcp.sse_app())

In [ ]:
if __name__ == "__main__":
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )

In [ ]:
python your_script.py

In [ ]:
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerSse

async def main():
    # Setup the SSE server parameters
    server_params = {"url": "http://localhost:3000/sse"}

    # Connect to the MCP server over SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        # Create an agent
        agent = Agent(
            name="OpenAI Shopping Agent",
            instructions="You are an assistant that uses shopping list tools to assist with a shopping list",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )
        
        # Run the agent
        result = await Runner.run(
            starting_agent=agent,
            input="Give me my shopping list"
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
# Mount the MCP SSE app at '/mcp'
app.mount('/mcp', mcp.sse_app())

In [ ]:
server_params = {"url": "http://localhost:3000/mcp/sse"}

# Exercise 1
You just learned how FastAPI and ASGI work together to build modern Python web applications. Now, let’s make sure you can run a basic FastAPI app on your own.

In this task, you'll start the provided FastAPI shopping list application and confirm that it works as expected. No code changes are needed for this step.

Follow these steps:

    Open your terminal.
    Navigate to the app directory.
    Run the FastAPI application using the command:
    python main.py
    Watch the terminal for a message indicating that the server is running on port 3000.

This will help you become comfortable with running FastAPI apps before you move on to integrating the MCP server.

In [ ]:
import uvicorn
from fastapi import FastAPI
from shopping_list import ShoppingListService

# Create the FastAPI app
app = FastAPI()

# Create a shopping list service instance
shopping_list = ShoppingListService()

# Regular FastAPI route for getting all shopping list items
@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

if __name__ == "__main__":
    # Run the server
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )

# Exercise 1
Now, let’s take the next step and connect your MCP server to the FastAPI application so that both regular endpoints and agent tools are available together within a single ASGI application.

Your task is to integrate the MCP server into the FastAPI app by:

    Importing the mcp_server module into the main application file.
    Mounting the MCP server’s SSE app at the '/' path using FastAPI’s app.mount() method.

There’s no need to manually start the FastAPI app—it’s already running in debug mode for you. The agent code is also preconfigured with the correct URL to connect to your MCP server. Once you’ve made your changes, simply click the Run button. The agent will automatically connect to your mounted MCP server (just like it would to any SSE server) and should be able to use the tools.

In [ ]:
import uvicorn
from fastapi import FastAPI
from shopping_list import ShoppingListService
# TODO: Import the MCP server module
from agents.mcp import MCPServerSse
from mcp_server import mcp

# Create the FastAPI app
app = FastAPI()

# Create a shopping list service instance
shopping_list = ShoppingListService()


# Regular FastAPI route for getting all shopping list items
@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

# TODO: Mount the MCP SSE app at the root path
app.mount('/', mcp.sse_app())

if __name__ == "__main__":
    # Run the server
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )

# Exercise 2
In this task, the MCP server is all set and mounted inside a FastAPI app, so both your shopping list tools and regular API endpoints are ready to go.

However, when we try running the agent script, the handshake with the MCP server doesn’t go through as expected. Take a close look at the provided code, spot what’s off in the agent’s configuration, and fix it so the agent can connect, communicate, and use the shopping list tools just like it should.

In [ ]:
# agent.py
import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerSse


async def main():
    # Setup the SSE server parameters
    server_params = {"url": "http://localhost:3000/sse"} # adjusted url to include /sse at end

    # Connect to the MCP server over SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        # Create an agent
        agent = Agent(
            name="OpenAI Shopping Agent",
            instructions="You are an assistant that uses shopping list tools to assist with a shopping list",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )
        
        # Run the agent
        result = await Runner.run(
            starting_agent=agent,
            input="Give me my shopping list"
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
# main.py
import uvicorn
from fastapi import FastAPI
from shopping_list import ShoppingListService
from mcp_server import mcp  # Import the MCP server with tools

# Create the FastAPI app
app = FastAPI()

# Create a shopping list service instance
shopping_list = ShoppingListService()


# Regular FastAPI route for getting all shopping list items
@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

# Mount the MCP SSE app at '/'
app.mount('/', mcp.sse_app())

if __name__ == "__main__":
    # Run the server
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )

# final exercise
Imagine you're building a growing application that needs to serve both regular API endpoints and agent tools. As your app expands, you want to keep things organized by placing different services in their own dedicated paths.

Your task is to:

    Mount the MCP server's SSE app at the /mcp path instead of the root path in your FastAPI application.
    Update the agent's connection URL so that it matches the new location of the MCP server (/mcp/sse).

This customization will help you maintain a clean architecture as your application grows, allowing you to add more services alongside your agent tools without path conflicts. It's a common practice in production environments where multiple services need to coexist under the same domain.s.

In [ ]:
# agent.py

import asyncio
from agents import Agent, Runner
from agents.mcp import MCPServerSse


async def main():
    # TODO: Update the server URL to match the new MCP server mount path
    server_params = {"url": "http://localhost:3000/mcp/sse"}

    # Connect to the MCP server over SSE
    async with MCPServerSse(params=server_params) as mcp_server:
        # Create an agent
        agent = Agent(
            name="OpenAI Shopping Agent",
            instructions="You are an assistant that uses shopping list tools to assist with a shopping list",
            mcp_servers=[mcp_server],
            model="gpt-4.1"
        )
        
        # Run the agent
        result = await Runner.run(
            starting_agent=agent,
            input="Give me my shopping list"
        )

        # Print the final output
        print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

In [ ]:
# main.py

import uvicorn
from fastapi import FastAPI
from shopping_list import ShoppingListService
from mcp_server import mcp

# Create the FastAPI app
app = FastAPI()

# Create a shopping list service instance
shopping_list = ShoppingListService()


# Regular FastAPI route for getting all shopping list items
@app.get("/items/")
async def get_all_items():
    """Get all shopping list items."""
    return shopping_list.get_items()

# TODO: Mount the MCP SSE app at '/mcp' instead of the root path
app.mount('/mcp', mcp.sse_app())

if __name__ == "__main__":
    # Run the server
    uvicorn.run(
        "main:app",
        host="0.0.0.0",
        port=3000,
        reload=True
    )